# Introdução
Este notebook é dedicado à feature engineering da tabela Credit Card Balance. Neste contexto, estarei explorando e transformando os dados da tabela para otimizar o desempenho dos modelos de machine learning que serão aplicados posteriormente. Feature engineering é uma etapa crucial no processo de preparação de dados, onde buscamos extrair informações relevantes e criar novas variáveis (ou features) que possam melhor capturar os padrões subjacentes aos nossos dados.

No contexto específico do nosso negócio de concessão de crédito, a feature engineering desempenha um papel fundamental. A capacidade de prever com precisão a probabilidade de inadimplência ou de pagamento pontual é crucial para a saúde financeira da instituição. Nesse sentido, a feature engineering permite não apenas melhorar a eficácia dos modelos de machine learning em realizar essas previsões, mas também oferece uma oportunidade de compreender melhor o comportamento dos clientes e os fatores que influenciam sua capacidade de pagamento.

O objetivo principal da feature engineering é, portanto, melhorar a capacidade dos modelos de machine learning em aprender com os dados, aumentando assim sua eficácia na realização de previsões ou classificações. Isso é alcançado através da seleção, transformação e criação de features que possam fornecer insights mais significativos e representativos para o problema em questão. Ao final deste processo, que será feito em todas as tabelas, esperamos obter um conjunto de dados mais refinado e adequado para alimentar nossos modelos de machine learning, resultando em predições mais precisas e confiáveis, o que é essencial para a tomada de decisões assertivas no âmbito da concessão de crédito.

## Configuração do Ambiente para Utilização do Spark
Configuração do ambiente e ferramentas essenciais que foram instaladas para o início do projeto.

Instalação do Apache Spark: O Apache Spark, uma poderosa ferramenta para processamento distribuído de dados em grande escala, foi instalado no ambiente de desenvolvimento. Esta instalação permitirá a utilização de todas as funcionalidades oferecidas pelo Spark para análise e manipulação de dados.

Configuração do Ambiente Python: Para interagir de forma eficiente com o Spark utilizando a linguagem Python, foram realizadas configurações específicas do ambiente Python. Isso inclui a instalação do pacote PySpark, uma biblioteca Python que fornece uma API para interagir com o Spark.

Criação da Sessão Spark: Uma sessão Spark foi criada utilizando a classe SparkSession. Essa sessão é fundamental para estabelecer uma conexão com o ambiente Spark e executar operações de processamento de dados distribuídas. Através dessa sessão, será possível acessar todos os recursos e funcionalidades do Spark para realizar análises e manipulações nos dados do projeto.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Minha Primeira Aplicação no Pyspark") \
        .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

##Leitura dos Dados

In [ ]:
df_credit_card_balance = spark.read.csv ('/content/drive/MyDrive/Colab Notebooks/AnaliseCredito/credit_card_balance.csv', header=True, inferSchema=True)
df_credit_card_balance.createOrReplaceTempView("df_credit_card_balance")

In [ ]:
df_credit_card_balance.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- AMT_BALANCE: double (nullable = true)
 |-- AMT_CREDIT_LIMIT_ACTUAL: integer (nullable = true)
 |-- AMT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |-- AMT_INST_MIN_REGULARITY: double (nullable = true)
 |-- AMT_PAYMENT_CURRENT: double (nullable = true)
 |-- AMT_PAYMENT_TOTAL_CURRENT: double (nullable = true)
 |-- AMT_RECEIVABLE_PRINCIPAL: double (nullable = true)
 |-- AMT_RECIVABLE: double (nullable = true)
 |-- AMT_TOTAL_RECEIVABLE: double (nullable = true)
 |-- CNT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_CURRENT: integer (nullable = true)
 |-- CNT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |

In [ ]:
df_credit_card_balance.show()

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+-----------+------------------

In [ ]:
# Contar o número de linhas
num_linhas = df_credit_card_balance.count()

# Contar o número de colunas
num_colunas = len(df_credit_card_balance.columns)

print("Número de linhas:", num_linhas)
print("Número de colunas:", num_colunas)

Número de linhas: 241596
Número de colunas: 23


##Criação de Flags para Visão Temporal dos Dados

Criação de flags para representar o comportamento do cliente ao longo de diferentes períodos de tempo em relação aos seus pagamentos.

In [ ]:
df_flag_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS ULT_3_MESES,
    CASE
        WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS ULT_6_MESES,
    CASE
        WHEN MONTHS_BALANCE >= -9 THEN 1
        ELSE 0
    END AS ULT_9_MESES,
    CASE
        WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS ULT_12_MESES,
    CASE
        WHEN MONTHS_BALANCE >= -18 THEN 1
        ELSE 0
    END AS ULT_18_MESES
FROM
    df_credit_card_balance
ORDER BY
    SK_ID_PREV;
""")
df_flag_01.createOrReplaceTempView("df_flag_01")
df_flag_01.show()

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+-----------+-----------+-----------+------------+------------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|ULT_3_M

##Criação de Feature Engineerings

Calculando estatísticas agregadas dos saldos de crédito ao longo de diferentes períodos para cada identificador de crédito anterior

In [ ]:
df_temp_01 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MED_AMT_BALANCE_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MIN_AMT_BALANCE_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MAX_AMT_BALANCE_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_SUM_AMT_BALANCE_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MED_AMT_BALANCE_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MIN_AMT_BALANCE_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MAX_AMT_BALANCE_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_SUM_AMT_BALANCE_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MED_AMT_BALANCE_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MIN_AMT_BALANCE_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MAX_AMT_BALANCE_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_SUM_AMT_BALANCE_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MED_AMT_BALANCE_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MIN_AMT_BALANCE_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MAX_AMT_BALANCE_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_SUM_AMT_BALANCE_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MED_AMT_BALANCE_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MIN_AMT_BALANCE_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_MAX_AMT_BALANCE_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN AMT_BALANCE ELSE NULL END) AS VL_SUM_AMT_BALANCE_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_01.createOrReplaceTempView("df_temp_01")
df_temp_01.show()

+----------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+
|SK_ID_PREV|VL_MED_AMT_BALANCE_ULT_3_MS|VL_MIN_AMT_BALANCE_ULT_3_MS|VL_MAX_AMT_BALANCE_ULT_3_MS|VL_SUM_AMT_BALANCE_ULT_3_MS|VL_MED_AMT_BALANCE_ULT_6_MS|VL_MIN_AMT_BALANCE_ULT_6_MS|VL_MAX_AMT_BALANCE_ULT_6_MS|VL_SUM_AMT_BALANCE_ULT_6_MS|VL_MED_AMT_BALANCE_ULT_9_MS|VL_MIN_AMT_BALANCE_ULT_9_MS|VL_MAX_AMT_BALANCE_ULT_9_MS|VL_SUM_AMT_BALANCE_ULT_9_MS|VL_MED_AMT_BALANCE_ULT_12_MS|VL_MIN_AMT_BALANCE_ULT_12_MS|VL_MAX_AMT_BA

Calculando métricas relacionadas ao limite de crédito dos clientes ao longo de diferentes períodos de tempo.

In [ ]:
df_temp_02 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MED_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MIN_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_SUM_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS,
    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MED_AMT_CREDIT_LIMIT_ACTUAL_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MIN_AMT_CREDIT_LIMIT_ACTUAL_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_SUM_AMT_CREDIT_LIMIT_ACTUAL_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MED_AMT_CREDIT_LIMIT_ACTUAL_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MIN_AMT_CREDIT_LIMIT_ACTUAL_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_SUM_AMT_CREDIT_LIMIT_ACTUAL_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MED_AMT_CREDIT_LIMIT_ACTUAL_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MIN_AMT_CREDIT_LIMIT_ACTUAL_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_SUM_AMT_CREDIT_LIMIT_ACTUAL_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MED_AMT_CREDIT_LIMIT_ACTUAL_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MIN_AMT_CREDIT_LIMIT_ACTUAL_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN AMT_CREDIT_LIMIT_ACTUAL ELSE NULL END) AS VL_SUM_AMT_CREDIT_LIMIT_ACTUAL_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_02.createOrReplaceTempView("df_temp_02")
df_temp_02.show()

+----------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|SK_ID_PREV|VL_MED_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS|VL_MIN_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS|VL_MAX_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS|VL_SUM_AMT_CREDIT_LIMIT_ACTUAL_ULT_3_MS|VL_MED_


Calculando métricas relacionadas aos saques em caixas eletrônicos realizados pelos clientes ao longo de diferentes períodos de tempo

In [ ]:
df_temp_03 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_ATM_CURRENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_ATM_CURRENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_ATM_CURRENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_ATM_CURRENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_ATM_CURRENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_ATM_CURRENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_ATM_CURRENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_ATM_CURRENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_ATM_CURRENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_ATM_CURRENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_ATM_CURRENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_ATM_CURRENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_ATM_CURRENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_ATM_CURRENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_ATM_CURRENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_ATM_CURRENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_ATM_CURRENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_ATM_CURRENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_ATM_CURRENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_ATM_CURRENT_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_03.createOrReplaceTempView("df_temp_03")
df_temp_03.show()

+----------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+
|SK_ID_PREV|VL_MED_AMT_DRAWINGS_ATM_CURRENT_ULT_3_MS|VL_MIN_AMT_DRAWINGS_ATM_CURRENT_ULT_3_MS|VL_MAX_AMT_DRAWINGS_ATM_CURRENT_ULT_3_MS|VL_SUM_AMT_DRAWINGS_ATM_


Calculando métricas relacionadas aos saques realizados pelos clientes ao longo de diferentes períodos de tempo

In [ ]:
df_temp_04 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_CURRENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_CURRENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_CURRENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_CURRENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_CURRENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_CURRENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_CURRENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_CURRENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_CURRENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_CURRENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_CURRENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_CURRENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_CURRENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_CURRENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_CURRENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_CURRENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_CURRENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_CURRENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_CURRENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_CURRENT_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_04.createOrReplaceTempView("df_temp_04")
df_temp_04.show()

+----------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+
|SK_ID_PREV|VL_MED_AMT_DRAWINGS_CURRENT_ULT_3_MS|VL_MIN_AMT_DRAWINGS_CURRENT_ULT_3_MS|VL_MAX_AMT_DRAWINGS_CURRENT_ULT_3_MS|VL_SUM_AMT_DRAWINGS_CURRENT_ULT_3_MS|VL_MED_AMT_DRAWINGS_CURRENT_ULT_6_MS|VL_MIN_AMT_DRAWINGS_CURRENT_ULT_6_MS|VL_MA


Calculando métricas relacionadas aos saques realizados em outras categorias pelos clientes ao longo de diferentes períodos de tempo.

In [ ]:
df_temp_05 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_OTHER_CURRENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_OTHER_CURRENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_OTHER_CURRENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_OTHER_CURRENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_OTHER_CURRENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_OTHER_CURRENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_OTHER_CURRENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_OTHER_CURRENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_OTHER_CURRENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_OTHER_CURRENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_OTHER_CURRENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_OTHER_CURRENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_OTHER_CURRENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_OTHER_CURRENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_OTHER_CURRENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_OTHER_CURRENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_OTHER_CURRENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_OTHER_CURRENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_OTHER_CURRENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN AMT_DRAWINGS_OTHER_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_OTHER_CURRENT_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_05.createOrReplaceTempView("df_temp_05")
df_temp_05.show()

+----------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+
|SK_ID_PREV|VL_MED_AMT_DRAWINGS_OTHER_CURRENT_ULT_3_MS|VL_MIN_AMT_DRAWINGS_OTHER_CURRENT_ULT_3_MS|VL_MAX_AMT_DRAWINGS_O


Calculando métricas relacionadas à regularidade mínima das parcelas pagas pelos clientes ao longo de diferentes períodos de tempo.

In [ ]:
df_temp_06 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MED_AMT_INST_MIN_REGULARITY_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MIN_AMT_INST_MIN_REGULARITY_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MAX_AMT_INST_MIN_REGULARITY_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_SUM_AMT_INST_MIN_REGULARITY_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MED_AMT_INST_MIN_REGULARITY_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MIN_AMT_INST_MIN_REGULARITY_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MAX_AMT_INST_MIN_REGULARITY_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_SUM_AMT_INST_MIN_REGULARITY_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MED_AMT_INST_MIN_REGULARITY_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MIN_AMT_INST_MIN_REGULARITY_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MAX_AMT_INST_MIN_REGULARITY_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_SUM_AMT_INST_MIN_REGULARITY_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MED_AMT_INST_MIN_REGULARITY_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MIN_AMT_INST_MIN_REGULARITY_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MAX_AMT_INST_MIN_REGULARITY_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_SUM_AMT_INST_MIN_REGULARITY_ULT_12_MS,

    AVG(CASE WHEN  ULT_18_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MED_AMT_INST_MIN_REGULARITY_ULT_18_MS,
    MIN(CASE WHEN  ULT_18_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MIN_AMT_INST_MIN_REGULARITY_ULT_18_MS,
    MAX(CASE WHEN  ULT_18_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_MAX_AMT_INST_MIN_REGULARITY_ULT_18_MS,
    SUM(CASE WHEN  ULT_18_MESES = 1 THEN AMT_INST_MIN_REGULARITY ELSE NULL END) AS VL_SUM_AMT_INST_MIN_REGULARITY_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_06.createOrReplaceTempView("df_temp_06")
df_temp_06.show()

+----------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|SK_ID_PREV|VL_MED_AMT_INST_MIN_REGULARITY_ULT_3_MS|VL_MIN_AMT_INST_MIN_REGULARITY_ULT_3_MS|VL_MAX_AMT_INST_MIN_REGULARITY_ULT_3_MS|VL_SUM_AMT_INST_MIN_REGULARITY_ULT_3_MS|VL_MED_


Calculando métricas relacionadas aos saques de valores durante o mês pelos clientes em diferentes períodos de tempo.

In [ ]:
df_temp_07 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_POS_CURRENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_POS_CURRENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_POS_CURRENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_POS_CURRENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_POS_CURRENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_POS_CURRENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_POS_CURRENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_POS_CURRENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_POS_CURRENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_POS_CURRENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_POS_CURRENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_POS_CURRENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_POS_CURRENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_POS_CURRENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_POS_CURRENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_POS_CURRENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MED_AMT_DRAWINGS_POS_CURRENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MIN_AMT_DRAWINGS_POS_CURRENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_MAX_AMT_DRAWINGS_POS_CURRENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES  = 1 THEN AMT_DRAWINGS_POS_CURRENT ELSE NULL END) AS VL_SUM_AMT_DRAWINGS_POS_CURRENT_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_07.createOrReplaceTempView("df_temp_07")
df_temp_07.show()

+----------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+
|SK_ID_PREV|VL_MED_AMT_DRAWINGS_POS_CURRENT_ULT_3_MS|VL_MIN_AMT_DRAWINGS_POS_CURRENT_ULT_3_MS|VL_MAX_AMT_DRAWINGS_POS_CURRENT_ULT_3_MS|VL_SUM_AMT_DRAWINGS_POS_


Calculando métricas relacionadas aos pagamentos efetuados pelos clientes em diferentes períodos de tempo.

In [ ]:
df_temp_08 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_CURRENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_CURRENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_CURRENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_CURRENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_CURRENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_CURRENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_CURRENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_CURRENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_CURRENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_CURRENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_CURRENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_CURRENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES  = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_CURRENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES  = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_CURRENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES  = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_CURRENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES  = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_CURRENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_CURRENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_CURRENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_CURRENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_CURRENT_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_08.createOrReplaceTempView("df_temp_08")
df_temp_08.show()

+----------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|SK_ID_PREV|VL_MED_AMT_PAYMENT_CURRENT_ULT_3_MS|VL_MIN_AMT_PAYMENT_CURRENT_ULT_3_MS|VL_MAX_AMT_PAYMENT_CURRENT_ULT_3_MS|VL_SUM_AMT_PAYMENT_CURRENT_ULT_3_MS|VL_MED_AMT_PAYMENT_CURRENT_ULT_6_MS|VL_MIN_AMT_PAYMENT_CURRENT_ULT_6_MS|VL_MAX_AMT_PAYMENT_CURRENT_ULT_

Calculando métricas relacionadas aos pagamentos totais efetuados pelos clientes em diferentes períodos de tempo.

In [ ]:
df_temp_09 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES  = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_TOTAL_CURRENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES  = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_TOTAL_CURRENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES  = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_TOTAL_CURRENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES  = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_TOTAL_CURRENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_TOTAL_CURRENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_TOTAL_CURRENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_TOTAL_CURRENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_TOTAL_CURRENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_TOTAL_CURRENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_TOTAL_CURRENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_TOTAL_CURRENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_TOTAL_CURRENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_TOTAL_CURRENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_TOTAL_CURRENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_TOTAL_CURRENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_TOTAL_CURRENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MED_AMT_PAYMENT_TOTAL_CURRENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MIN_AMT_PAYMENT_TOTAL_CURRENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_MAX_AMT_PAYMENT_TOTAL_CURRENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN AMT_PAYMENT_TOTAL_CURRENT ELSE NULL END) AS VL_SUM_AMT_PAYMENT_TOTAL_CURRENT_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_09.createOrReplaceTempView("df_temp_09")
df_temp_09.show()

+----------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+
|SK_ID_PREV|VL_MED_AMT_PAYMENT_TOTAL_CURRENT_ULT_3_MS|VL_MIN_AMT_PAYMENT_TOTAL_CURRENT_ULT_3_MS|VL_MAX_AMT_PAYMENT_TOTAL_CURRENT_ULT_3_MS|V

Calculando métricas relacionadas ao número de saques em caixas eletrônicos realizados pelos clientes em diferentes períodos de tempo.

In [ ]:
df_temp_10 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_CNT_DRAWINGS_ATM_CURRENT_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_CNT_DRAWINGS_ATM_CURRENT_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_CNT_DRAWINGS_ATM_CURRENT_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_CNT_DRAWINGS_ATM_CURRENT_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_CNT_DRAWINGS_ATM_CURRENT_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_CNT_DRAWINGS_ATM_CURRENT_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_CNT_DRAWINGS_ATM_CURRENT_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_CNT_DRAWINGS_ATM_CURRENT_ULT_6_MS,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_CNT_DRAWINGS_ATM_CURRENT_ULT_9_MS,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_CNT_DRAWINGS_ATM_CURRENT_ULT_9_MS,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_CNT_DRAWINGS_ATM_CURRENT_ULT_9_MS,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_CNT_DRAWINGS_ATM_CURRENT_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_CNT_DRAWINGS_ATM_CURRENT_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_CNT_DRAWINGS_A_CURRENT_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_CNT_DRAWINGS_ATM_CURRENT_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_CNT_DRAWINGS_ATM_CURRENT_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MED_CNT_DRAWINGS_ATM_CURRENT_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MIN_CNT_DRAWINGS_ATM_CURRENT_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_MAX_CNT_DRAWINGS_ATM_CURRENT_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES  = 1 THEN CNT_DRAWINGS_ATM_CURRENT ELSE NULL END) AS VL_SUM_CNT_DRAWINGS_ATM_CURRENT_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_10.createOrReplaceTempView("df_temp_10")
df_temp_10.show()

+----------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+-----------------------------------------+---------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+
|SK_ID_PREV|VL_MED_CNT_DRAWINGS_ATM_CURRENT_ULT_3_MS|VL_MIN_CNT_DRAWINGS_ATM_CURRENT_ULT_3_MS|VL_MAX_CNT_DRAWINGS_ATM_CURRENT_ULT_3_MS|VL_SUM_CNT_DRAWINGS_ATM_CU

Calculando métricas relacionadas ao número de parcelas pagas pelos clientes ao longo de diferentes períodos de tempo.

In [ ]:
df_temp_11 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MED_CNT_INSTALMENT_MATURE_CUM_ULT_3_MS,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MIN_CNT_INSTALMENT_MATURE_CUM_ULT_3_MS,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MAX_CNT_INSTALMENT_MATURE_CUM_ULT_3_MS,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_SUM_CNT_INSTALMENT_MATURE_CUM_ULT_3_MS,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MED_CNT_INSTALMENT_MATURE_CUM_ULT_6_MS,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MIN_CNT_INSTALMENT_MATURE_CUM_ULT_6_MS,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MAX_CNT_INSTALMENT_MATURE_CUM_ULT_6_MS,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_SUM_CNT_INSTALMENT_MATURE_CUM_ULT_6_MS,

    AVG(CASE WHEN  ULT_9_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MED_CNT_INSTALMENT_MATURE_CUM_ULT_9_MS,
    MIN(CASE WHEN  ULT_9_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MIN_CNT_INSTALMENT_MATURE_CUM_ULT_9_MS,
    MAX(CASE WHEN  ULT_9_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MAX_CNT_INSTALMENT_MATURE_CUM_ULT_9_MS,
    SUM(CASE WHEN  ULT_9_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_SUM_CNT_INSTALMENT_MATURE_CUM_ULT_9_MS,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MED_CNT_INSTALMENT_MATURE_CUM_ULT_12_MS,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MIN_CNT_INSTALMENT_MATURE_CUM_ULT_12_MS,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MAX_CNT_INSTALMENT_MATURE_CUM_ULT_12_MS,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_SUM_CNT_INSTALMENT_MATURE_CUM_ULT_12_MS,

    AVG(CASE WHEN ULT_18_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MED_CNT_INSTALMENT_MATURE_CUM_ULT_18_MS,
    MIN(CASE WHEN ULT_18_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MIN_CNT_INSTALMENT_MATURE_CUM_ULT_18_MS,
    MAX(CASE WHEN ULT_18_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_MAX_CNT_INSTALMENT_MATURE_CUM_ULT_18_MS,
    SUM(CASE WHEN ULT_18_MESES  = 1 THEN CNT_INSTALMENT_MATURE_CUM ELSE NULL END) AS VL_SUM_CNT_INSTALMENT_MATURE_CUM_ULT_18_MS
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_11.createOrReplaceTempView("df_temp_11")
df_temp_11.show()

+----------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+
|SK_ID_PREV|VL_MED_CNT_INSTALMENT_MATURE_CUM_ULT_3_MS|VL_MIN_CNT_INSTALMENT_MATURE_CUM_ULT_3_MS|VL_MAX_CNT_INSTALMENT_MATURE_CUM_ULT_3_MS|V

Calculando métricas relacionadas ao atraso no pagamento do saldo devedor em cartão de crédito ao longo de diferentes períodos de tempo.

In [ ]:
df_temp_12 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MED_SK_DPD_ULT_3_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MIN_SK_DPD_ULT_3_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MAX_SK_DPD_ULT_3_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_SUM_SK_DPD_ULT_3_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MED_SK_DPD_ULT_6_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MIN_SK_DPD_ULT_6_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MAX_SK_DPD_ULT_6_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_SUM_SK_DPD_ULT_6_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MED_SK_DPD_ULT_9_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MIN_SK_DPD_ULT_9_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MAX_SK_DPD_ULT_9_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_SUM_SK_DPD_ULT_9_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MED_SK_DPD_ULT_12_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MIN_SK_DPD_ULT_12_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MAX_SK_DPD_ULT_12_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_SUM_SK_DPD_ULT_12_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MED_SK_DPD_ULT_18_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MIN_SK_DPD_ULT_18_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_MAX_SK_DPD_ULT_18_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD ELSE NULL END) AS VL_SUM_SK_DPD_ULT_18_MS_CREDIT_CARD
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_12.createOrReplaceTempView("df_temp_12")
df_temp_12.show()

+----------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+
|SK_ID_PREV|VL_MED_SK_DPD_ULT_3_MS_CREDIT_CARD|VL_MIN_SK_DPD_ULT_3_MS_CREDIT_CARD|VL_MAX_SK_DPD_ULT_3_MS_CREDIT_CARD|VL_SUM_SK_DPD_ULT_3_MS_CREDIT_CARD|VL_MED_SK_DPD_ULT_6_MS_CREDIT_CARD|VL_MIN_SK_DPD_ULT_6_MS_CREDIT_CARD|VL_MAX_SK_DPD_ULT_6_MS_CREDIT_CARD|VL_SUM_SK_DPD_ULT_6_MS

Calculando métricas relacionadas ao atraso no pagamento do saldo devedor em cartão de crédito, considerando o status de atraso definitivo, ao longo de diferentes períodos de tempo.

In [ ]:
df_temp_13 = spark.sql("""
  SELECT
    SK_ID_PREV,
    AVG(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MED_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MIN_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MAX_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_3_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_SUM_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MED_SK_DPD_DEF_ULT_6_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MIN_SK_DPD_DEF_ULT_6_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MAX_SK_DPD_DEF_ULT_6_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_6_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_SUM_SK_DPD_DEF_ULT_6_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MED_SK_DPD_DEF_ULT_9_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MIN_SK_DPD_DEF_ULT_9_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MAX_SK_DPD_DEF_ULT_9_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_9_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_SUM_SK_DPD_DEF_ULT_9_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MED_SK_DPD_DEF_ULT_12_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MIN_SK_DPD_DEF_ULT_12_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MAX_SK_DPD_DEF_ULT_12_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_12_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_SUM_SK_DPD_DEF_ULT_12_MS_CREDIT_CARD,

    AVG(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MED_SK_DPD_DEF_ULT_18_MS_CREDIT_CARD,
    MIN(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MIN_SK_DPD_DEF_ULT_18_MS_CREDIT_CARD,
    MAX(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_MAX_SK_DPD_DEF_ULT_18_MS_CREDIT_CARD,
    SUM(CASE WHEN ULT_18_MESES = 1 THEN SK_DPD_DEF ELSE NULL END) AS VL_SUM_SK_DPD_DEF_ULT_18_MS_CREDIT_CARD
FROM
    df_flag_01
GROUP BY
    SK_ID_PREV
ORDER BY
    SK_ID_PREV;
   """)
df_temp_13.createOrReplaceTempView("df_temp_13")
df_temp_13.show()

+----------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+
|SK_ID_PREV|VL_MED_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD|VL_MIN_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD|VL_MAX_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD|VL_SUM_SK_DPD_DEF_ULT_3_MS_CREDIT_CARD|VL_MED_SK_DPD_DEF_ULT_6_MS_CRED

##Realização de Joins entre DataFrames

O DataFrame resultante dessa operação de junção contém todas as informações agregadas e combinadas dos clientes.

In [ ]:
# Realizar o join encadeado
df_credit_card_balance_01 = df_credit_card_balance \
    .join(df_temp_01, on='SK_ID_PREV', how='inner') \
    .join(df_temp_02, on='SK_ID_PREV', how='inner') \
    .join(df_temp_03, on='SK_ID_PREV', how='inner') \
    .join(df_temp_04, on='SK_ID_PREV', how='inner') \
    .join(df_temp_05, on='SK_ID_PREV', how='inner') \
    .join(df_temp_06, on='SK_ID_PREV', how='inner') \
    .join(df_temp_07, on='SK_ID_PREV', how='inner') \
    .join(df_temp_08, on='SK_ID_PREV', how='inner') \
    .join(df_temp_09, on='SK_ID_PREV', how='inner') \
    .join(df_temp_10, on='SK_ID_PREV', how='inner') \
    .join(df_temp_11, on='SK_ID_PREV', how='inner') \
    .join(df_temp_12, on='SK_ID_PREV', how='inner') \
    .join(df_temp_13, on='SK_ID_PREV', how='inner')

# Exibir o DataFrame resultante
df_credit_card_balance_01.show()
df_credit_card_balance_01.createOrReplaceTempView("df_credit_card_balance_01")

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------

Renomeando as colunas SK_ID_CURR e NAME_CONTRACT_STATUS, com o objetivo de não haver ambiguidade nos joins com os outros dataframes.

In [ ]:
from pyspark.sql.functions import col

# Renomear as colunas
df_credit_card_balance_01 = df_credit_card_balance_01.withColumnRenamed('SK_ID_CURR', 'SK_ID_CURR_CREDIT_CARD') \
                                       .withColumnRenamed('NAME_CONTRACT_STATUS', 'NAME_CONTRACT_STATUS_CREDIT_CARD')\
                                       .withColumnRenamed('MONTHS_BALANCE', 'MONTHS_BALANCE_CREDIT_CARD')\
                                       .withColumnRenamed('SK_DPD', 'SK_DPD_CREDIT_CARD')\
                                       .withColumnRenamed('SK_DPD_DEF', 'SK_DPD_DEF_CREDIT_CARD')\


# Exibir o DataFrame resultante
df_credit_card_balance_01.show()


+----------+----------------------+--------------------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------------------+------------------+----------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------

Calculando a proporção de utilização do limite do cartão de crédito em relação ao saldo atual

In [ ]:
df_credit_card_balance_01.createOrReplaceTempView("df_credit_card_balance_01")
df_temp_14 = spark.sql("""
  SELECT *,
    ROUND(AMT_BALANCE / AMT_CREDIT_LIMIT_ACTUAL,2) AS UTILIZ_LIMITE_CARTAO
  FROM
    df_credit_card_balance_01;
""")
df_temp_14.show()
df_temp_14.createOrReplaceTempView("df_temp_14")

+----------+----------------------+--------------------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------------------+------------------+----------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------

Contando o número total de saques para cada SK_ID_PREV, considerando apenas os registros em que o valor do saque atual é maior que zero.

In [ ]:
df_temp_15 = spark.sql("""SELECT
    *,
    COUNT(*) OVER (PARTITION BY SK_ID_PREV) AS NUMERO_TOTAL_SAQUES
FROM
   df_temp_14
WHERE
    AMT_DRAWINGS_CURRENT > 0;
    """)
df_temp_15.show()
df_temp_15.createOrReplaceTempView("df_temp_15")

+----------+----------------------+--------------------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------------------+------------------+----------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------

Calculando a variação do saldo de crédito para cada linha em relação ao mês anterior, agrupado por SK_ID_PREV.

In [ ]:

df_credit_card_balance_final_01 = spark.sql("""
SELECT
    *,
    (AMT_BALANCE - LAG(AMT_BALANCE) OVER (PARTITION BY SK_ID_PREV ORDER BY MONTHS_BALANCE_CREDIT_CARD)) AS VARIACAO_SALDO_CREDITO
FROM
    df_temp_15;
    """)
df_credit_card_balance_final_01.show()
df_temp_15.createOrReplaceTempView("df_credit_card_balance_final_01")

+----------+----------------------+--------------------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------------------+------------------+----------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------

##Agregação de Variáveis na Visão Cliente

Realização de uma sumarização dos dados na perspectiva do cliente.

In [ ]:
from pyspark.sql.functions import first

# Lista de todas as colunas, exceto a coluna SK_ID_PREV
colunas_para_agregar = [col for col in df_credit_card_balance_final_01.columns if col != "SK_ID_PREV"]

# Agregue as variáveis por SK_ID_PREV
df_credit_card_balance_final_02 = df_credit_card_balance_final_01.groupBy("SK_ID_PREV").agg(
    *[first(col).alias(col) for col in colunas_para_agregar]
)

# Mostre o resultado
df_credit_card_balance_final_02.show()

+----------+----------------------+--------------------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------------------+------------------+----------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------

##Verificando a presença de colunas duplicadas

In [ ]:
# Verificar os nomes das colunas
nomes_colunas = df_credit_card_balance_final_02.columns

# Verificar ambiguidade
colunas_unicas = set(nomes_colunas)
if len(colunas_unicas) != len(nomes_colunas):
    print("A tabela contém colunas duplicadas.")
else:
    print("A tabela não contém colunas duplicadas.")

# Contar a quantidade de colunas
quantidade_colunas = len(nomes_colunas)
print("A quantidade de colunas na tabela é:", quantidade_colunas)

# Converter lista de colunas para tupla e conjunto (set)
tupla_colunas = tuple(nomes_colunas)
conjunto_colunas = set(nomes_colunas)

print("Lista de colunas em forma de tupla:", tupla_colunas)
print("Lista de colunas em forma de conjunto (set):", conjunto_colunas)

A tabela não contém colunas duplicadas.
A quantidade de colunas na tabela é: 286
Lista de colunas em forma de tupla: ('SK_ID_PREV', 'SK_ID_CURR_CREDIT_CARD', 'MONTHS_BALANCE_CREDIT_CARD', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS_CREDIT_CARD', 'SK_DPD_CREDIT_CARD', 'SK_DPD_DEF_CREDIT_CARD', 'VL_MED_AMT_BALANCE_ULT_3_MS', 'VL_MIN_AMT_BALANCE_ULT_3_MS', 'VL_MAX_AMT_BALANCE_ULT_3_MS', 'VL_SUM_AMT_BALANCE_ULT_3_MS', 'VL_MED_AMT_BALANCE_ULT_6_MS', 'VL_MIN_AMT_BALANCE_ULT_6_MS', 'VL_MAX_AMT_BALANCE_ULT_6_MS', 'VL_SUM_AMT_BALANCE_ULT_6_MS', 'VL_MED_AMT_BALANCE_ULT_9_MS', 'VL_MIN_AM

##Salvar a tabela Sumarizada

In [ ]:
df_credit_card_balance_final_02 = df_credit_card_balance_final_02.repartition(1)
df_credit_card_balance_final_02.write.mode("overwrite").csv("credit_card_balance_agg_01.csv",header=True)